# Add computational predictions to data

This notebook takes an input dataframe from step 1, adds in-silico predictions
made by various modeling engines, and outputs to a dataframe for OKS scoring.

In [ ]:
import pandas as pd

data = pd.read_pickle("../data/data_rdatOnly.pkl")

predictions = pd.read_csv('data/predictions/silico_predictions.csv', index_col="id", dtype=str)
predictions.drop(columns=["name","title","body","sequence"],inplace=True)

In [ ]:
data.index.astype(str)
predictions.index.astype(str)
merged = pd.merge(data,predictions,how="outer",on='id')
display(merged)

In [40]:
merged.to_pickle('../data/data+predictions.pkl')